In [1]:
!pip3 install torch torchvision torchaudio

In [15]:
import torch 
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
import os
import numpy as np
import nibabel as nib
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import re
from torchvision import transforms

class DeepLesionDataset(Dataset):
    def __init__(self, data_dir, train_csv=None, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        
        # Get the list of PNG files
        self.png_files = []
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".png"):
                    self.png_files.append(os.path.join(root, file))
        
        # Read the CSV file if provided
        if train_csv is not None:
            self.info = pd.read_csv(train_csv)
        else:
            self.info = None
        
        # Define the default transform to convert PIL images to tensors
        self.default_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ConvertImageDtype(torch.float),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0) == 1 else x)
        ])
    
    def __len__(self):
        return len(self.png_files)
    
    def __getitem__(self, index):
        # Get the PNG filename
        png_path = self.png_files[index]
        
        # Load PNG image
        png_image = Image.open(png_path)
        
        # Apply the default transform to convert PIL image to tensor
        png_image = self.default_transform(png_image)
        
        # Apply additional transforms if provided
        if self.transform:
            png_image = self.transform(png_image)
        
        # Extract patient, study, series, and slice indices from the filename
        parts = png_path.split(os.sep)  # Split the path using the appropriate separator
        
        if len(parts) >= 4:
            try:
                patient_idx = int(parts[-4])
                study_idx = int(parts[-3])
                series_idx = int(parts[-2])
                slice_idx = int(parts[-1].split(".")[0])
            except ValueError:
                patient_idx, study_idx, series_idx, slice_idx = 0, 0, 0, 0  # Set default values if parsing fails
        else:
            patient_idx, study_idx, series_idx, slice_idx = 0, 0, 0, 0  # Set default values if filename format is unexpected
        
        # Get the corresponding label from the CSV file if provided
        if self.info is not None and 'label' in self.info.columns:
            label = self.info.loc[index, 'label']
            label = torch.tensor(label, dtype=torch.long)
        else:
            label = torch.tensor(-1, dtype=torch.long)  # Set a default value for the label if not available
        
        return png_image, label, patient_idx, study_idx, series_idx, slice_idx

In [55]:
import torch
import torch.nn as nn
import torchvision.models as models

class UniversalLesionDetector(nn.Module):
    def __init__(self, num_classes):
        super(UniversalLesionDetector, self).__init__()
        self.num_classes = num_classes
        self.anchor_scales = [16, 24, 32, 48, 96]  # Example anchor scales
        self.anchor_ratios = [0.5, 1.0, 2.0]  # Example anchor ratios
        
        # Backbone network (VGG-16)
        vgg = models.vgg16(pretrained=True)
        self.features = nn.Sequential(*list(vgg.features.children())[:-2])
        
        # Region Proposal Network (RPN)
        self.rpn = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.rpn_cls = nn.Conv2d(512, 18, 1)
        self.rpn_reg = nn.Conv2d(512, 36, 1)
        
        # RoI Pooling
        self.roi_pool = nn.AdaptiveMaxPool2d(7)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 5, padding=0),
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(512, num_classes),
            nn.Linear(512, num_classes * 4)
        )
    
    def forward(self, x):
        # Extract features
        features = self.features(x)
        
        # Region Proposal Network
        rpn_output = self.rpn(features)
        rpn_cls_scores = self.rpn_cls(rpn_output)
        rpn_bbox_pred = self.rpn_reg(rpn_output)
        
        # Get the output size of the rpn_reg layer
        rpn_reg_output_size = rpn_bbox_pred.size()
        num_anchors = rpn_reg_output_size[1] // 4
        
        rpn_bbox_pred = rpn_bbox_pred.permute(0, 2, 3, 1).contiguous().view(rpn_bbox_pred.size(0), -1, 4)
        
        # Generate proposals
        proposals = self.generate_proposals(rpn_cls_scores, rpn_bbox_pred, num_anchors)
        
        # RoI Pooling
        roi_features = self.roi_pool(features, proposals)
        
        # Classifier
        cls_scores = self.classifier[:5](roi_features)
        cls_scores = cls_scores.view(cls_scores.size(0), -1)
        bbox_pred = self.classifier[5:](roi_features)
        bbox_pred = bbox_pred.view(bbox_pred.size(0), -1, 4)
        
        return cls_scores, bbox_pred
        
        return cls_scores, bbox_pred
    def generate_proposals(self, rpn_cls_scores, rpn_bbox_pred, num_anchors):
        # Generate anchors
        anchors = self.generate_anchors(num_anchors)
    
        # Apply NMS to filter out low-scoring and redundant proposals
        proposals, proposal_scores = self.nms(rpn_cls_scores, rpn_bbox_pred, anchors)
    
        return proposals, proposal_scores
    
        return proposals
    def generate_anchors(self, num_anchors):
        feature_map_size = (32, 32)  # Assuming input feature map size is (32, 32)
        num_anchors_per_location = num_anchors // (feature_map_size[0] * feature_map_size[1])
    
        anchors = []
        for scale in self.anchor_scales:
            for ratio in self.anchor_ratios:
                w = scale * torch.sqrt(torch.tensor(ratio, dtype=torch.float32))
                h = scale / torch.sqrt(torch.tensor(ratio, dtype=torch.float32))
                for x in range(feature_map_size[1]):
                    for y in range(feature_map_size[0]):
                        anchors.extend([[(x + 0.5) / feature_map_size[1], (y + 0.5) / feature_map_size[0], w, h]])
        anchors = torch.tensor(anchors, dtype=torch.float32)
        return anchors
    
    def nms(self, rpn_cls_scores, rpn_bbox_pred, anchors):
        batch_size = rpn_cls_scores.size(0)
        num_anchors = rpn_cls_scores.size(1)
        print(f"rpn_bbox_pred shape: {rpn_bbox_pred.shape}")  # Add this line
        # Convert RPN outputs to bounding boxes
        print(f"num anchors : {num_anchors}")
        # rpn_bbox_pred = rpn_bbox_pred.view(batch_size, num_anchors, 4)
        
        proposals = self.decode_bboxes(rpn_bbox_pred, anchors)
        
        # Apply NMS for each image in the batch
        proposals_list = []
        scores_list = []
        for i in range(batch_size):
            scores = rpn_cls_scores[i, :, 1]  # Foreground scores
            proposal_scores, proposal_indices = scores.sort(descending=True)
            proposal_boxes = proposals[i, proposal_indices, :]
            
            keep_indices = self.nms_helper(proposal_boxes, proposal_scores)
            proposals_list.append(proposal_boxes[keep_indices, :])
            scores_list.append(proposal_scores[keep_indices])
        
        proposals = torch.cat(proposals_list, dim=0)
        proposal_scores = torch.cat(scores_list, dim=0)
        
        return proposals, proposal_scores
    def nms_helper(self, boxes, scores, nms_thresh=0.7):
        # Apply non-maximum suppression
        keep_indices = torch.zeros(len(boxes), dtype=torch.bool)
        for i, box in enumerate(boxes):
            if keep_indices[i]: continue
            keep_indices[i] = True
            ious = self.compute_iou(box, boxes)
            keep_indices[ious > nms_thresh] = False
        return keep_indices.nonzero().squeeze(1)
    def decode_bboxes(self, rpn_bbox_pred, anchors):
        batch_size = rpn_bbox_pred.size(0)
        num_anchors = rpn_bbox_pred.size(1)
    
        # Convert RPN bounding box predictions to actual proposals
        proposals = torch.zeros((batch_size, num_anchors, 4), dtype=torch.float32, device=rpn_bbox_pred.device)
    
        anchors = anchors.repeat(batch_size, 1, 1)  # Repeat anchors to match the batch size
    
        proposals[:, :, 0] = rpn_bbox_pred[:, :, 0] * anchors[:, :, 2] + anchors[:, :, 0]  # x1
        proposals[:, :, 1] = rpn_bbox_pred[:, :, 1] * anchors[:, :, 3] + anchors[:, :, 1]  # y1
        proposals[:, :, 2] = torch.exp(rpn_bbox_pred[:, :, 2]) * anchors[:, :, 2] + anchors[:, :, 0]  # x2
        proposals[:, :, 3] = torch.exp(rpn_bbox_pred[:, :, 3]) * anchors[:, :, 3] + anchors[:, :, 1]  # y2
    
        return proposals
    
    def compute_iou(self, box1, boxes):
        # Convert box coordinates to (x1, y1, x2, y2) format
        box1 = box1.unsqueeze(0)
        x1, y1, x2, y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
    
        boxes_x1, boxes_y1, boxes_x2, boxes_y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    
        # Calculate intersection areas
        inter_x1 = torch.max(x1, boxes_x1)
        inter_y1 = torch.max(y1, boxes_y1)
        inter_x2 = torch.min(x2, boxes_x2)
        inter_y2 = torch.min(y2, boxes_y2)
    
        inter_area = torch.clamp(inter_x2 - inter_x1 + 1.0, min=0.0) * torch.clamp(inter_y2 - inter_y1 + 1.0, min=0.0)
    
        # Calculate union areas
        box1_area = (x2 - x1 + 1.0) * (y2 - y1 + 1.0)
        boxes_area = (boxes_x2 - boxes_x1 + 1.0) * (boxes_y2 - boxes_y1 + 1.0)
    
        union_area = box1_area + boxes_area - inter_area
    
        # Compute IoU
        iou = inter_area / union_area
    
        return iou

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd


#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
# Set the paths to the DeepLesion dataset
data_dir = './'
train_csv = 'DL_info.csv'  # CSV file with training data information

# Set the hyperparameters
num_classes = 2  # Number of classes (lesion vs. non-lesion)
batch_size = 4
num_epochs = 10
learning_rate = 0.001

# Create the dataset and data loader
train_dataset = DeepLesionDataset(data_dir, train_csv=train_csv, transform=None)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the model, loss function, and optimizer
model = UniversalLesionDetector(num_classes)
model.to(device)  # Move the model to the device
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels, patient_idx, study_idx, series_idx, slice_idx in train_loader:
        # Move the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

/home/nevdread/miniconda3/envs/phas/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nevdread/miniconda3/envs/phas/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


rpn_bbox_pred shape: torch.Size([4, 9216, 4])
num anchors : 18


RuntimeError: The size of tensor a (9216) must match the size of tensor b (15360) at non-singleton dimension 1